In [16]:
import os
import json

import pandas as pd

from tqdm import tqdm
#from arena_util import write_json
#from arena_util import remove_seen
#from arena_util import is_tag
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
from gensim.models import KeyedVectors

from collections import Counter
import sentencepiece as spm
import re
import io
import numpy as np
import distutils.dir_util

In [17]:
def write_json(data, FILE_PATH, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(FILE_PATH + parent)
    with io.open(FILE_PATH + fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

def remove_seen(seen, l):
    seen = set(seen)
    return [x for x in l if not (x in seen)]

###########arena_util 클래스에 아래 함수들 추가! ##########    
def is_tag(word,tags_list_all):
    if word in tags_list_all :
        return True
    else:
        return False

def before_updt_date(cand_song_idx,updt_date,song_meta):
            
    updt_date = int(re.sub('-','',updt_date)[:8])
    return_idx = []
    
    for i in cand_song_idx:
        if int(song_meta.loc[i,'issue_date'])>updt_date :
            continue
        else:
            return_idx.append(i)

    return return_idx

In [19]:
#train_addtags.json , val_addtags.json 읽어왔으면 실행안해도 됌!!

class MakeAdditionalTags:
    def __init__(self,FILE_PATH):
        self.FILE_PATH = FILE_PATH
        with open(os.path.join(FILE_PATH, 'genre_gn_all.json'), encoding="utf-8") as f:
            self.genre_gn_all = json.load(f)
        self.res = []
            

    def mk_genre_tags (self,songs_input,song_meta, genre_gn_all, tag_num):
        genre_list=[]
        genre_tags_list=[]
        
        for i in songs_input:
            genre_list+=song_meta.loc[i,'song_gn_gnr_basket']
        c = Counter(genre_list)
        genre_list=c.most_common(tag_num)
        
        for i in range(len(genre_list)):
            try:genre_names = self.genre_gn_all[genre_list[i][0]].split('/')
            except KeyError : continue
            genre_tags_list += genre_names
            
        return genre_tags_list
    
    def sentence_piece (self,train):
        titles_train = train[train['plylst_title'] != '']['plylst_title']

        f = open('titles.txt', mode='wt', encoding='utf-8')
        for i in titles_train : 
            f.write(re.sub(r'[^가-힣a-zA-Z0-9\s]','',i)+'\n')

        for tags in train['tags']:
            for tag in tags:
                f.write(tag)
                f.write(' ')
            f.write('\n')

        f.close()

        templates = '--input=titles.txt \
        --model_prefix=train \
        --vocab_size=20000 \
        --character_coverage=1.0 \
        --model_type=bpe \
        '

        spm.SentencePieceTrainer.Train(templates)
    
    def mk_title_tags (self,data,tag_list_all):
        sp = spm.SentencePieceProcessor()
        sp.load('train.model')

        sp_title = []
        for i in tqdm(data['plylst_title']) :
            i = re.sub(r'[^가-힣a-zA-Z0-9\s]','',i)
            if type(i) != str : sp_title.append([])
            else:
                pieces = sp.encode_as_pieces(i)
                plus_tag = []
                for i in pieces:
                    tag = re.sub('▁','',i)

                    if (len(tag) > 1) and (tag[-1] == '과' or tag[-1] == '와'):
                        tag = tag[:-1]

                    if is_tag(tag,tag_list_all):
                        plus_tag.append(tag)
                sp_title.append(plus_tag)

        data['title_tags'] = sp_title
    
    def run (self,data,fname):
        data['genre_tags'] = data.apply(lambda x : self.mk_genre_tags(x['songs'],song_meta,self.genre_gn_all,2),axis=1)
        tags_list_all = []
        for tags in train['tags']:
            tags_list_all+=tags
        for tags in val['tags']:
            tags_list_all+=tags
        tags_list_all = list(set(tags_list_all))
        
        self.mk_title_tags(data,tags_list_all)
        

        #태그 추가한 val.json 데이터 새로 쓰기
        for pid in data.index:
            self.res.append({
                        "tags":data.loc[pid,"tags"],
                        "id": data.loc[pid, "id"],
                        "songs": data.loc[pid, "songs"],
                        "plylst_title": data.loc[pid, "plylst_title"],
                        "like_cnt": data.loc[pid, "like_cnt"],
                        "updt_date": data.loc[pid, "updt_date"],
                        "genre_tags": data.loc[pid, "genre_tags"],
                        "title_tags": data.loc[pid, "title_tags"]
                })

        write_json(self.res, self.FILE_PATH, fname+'_addtags.json')


In [23]:
'''
FILE_PATH = ''
val_tags = MakeAdditionalTags(FILE_PATH)
val_tags.run(val,'val')
train_tags = MakeAdditionalTags(FILE_PATH)
train_tags.run(train,'train')
'''
val = pd.read_json('val.json',typ = 'frame',encoding='utf8')

train = pd.read_json('train.json',typ = 'frame',encoding='utf8')
song_meta = pd.read_json('song_meta.json',typ = 'frame',encoding='utf8')
test = pd.read_json('test.json',typ='frame',encoding='utf8')
test_tags = MakeAdditionalTags(FILE_PATH)
test_tags.run(test,'test')

100%|██████████| 10740/10740 [00:06<00:00, 1673.50it/s]


In [24]:
#from arena_util import before_updt_date
#from AreanaUtil import remove_seen

class MakeBaselineResults:
    
    def __init__(self, FILE_PATH):
        self.FILE_PATH = FILE_PATH
        
        with open(os.path.join(FILE_PATH, 'train_addtags.json'), encoding="utf-8") as f:
            self.train = pd.DataFrame(json.load(f))
        
        with open(os.path.join(FILE_PATH, 'val_addtags.json'), encoding="utf-8") as f:
            self.val = pd.DataFrame(json.load(f))
          
        with open(os.path.join(FILE_PATH, 'test_addtags.json'), encoding="utf-8") as f:
            self.test = pd.DataFrame(json.load(f))
        
        with open(os.path.join(FILE_PATH, 'song_meta.json'), encoding="utf-8") as f:
            self.song_meta = pd.DataFrame(json.load(f))
        
        
    
    def intersect_cnt(self,tags,cand_tags,title_tags,genre_tags):
        
        tags_o = len(list(set(tags)&set(cand_tags))) * 3
        tags_t = len(list(set(tags)&set(title_tags))) * 2
        tags_g = len(list(set(tags)&set(genre_tags)))

        score = tags_o+tags_t+tags_g

        return score

    def mk_rec(self, x):
        t = pd.concat([self.train, self.val,self.test], ignore_index = True)
        t['cnt'] = 0

        tags_input = x.tags.copy()
        tags_genre = x.genre_tags.copy()
        tags_title = x.title_tags.copy()

        tag_score = t['tags'].apply(lambda x : self.intersect_cnt(x,tags_input,tags_title,tags_genre))
        t['cnt'] += tag_score


        t = t.sort_values(by='cnt',ascending=False)

        max_cnt = t.cnt.values[0]

        tag_result = []
        song_result = []

        while max_cnt > 0  :

            tl = list(t[t['cnt'] == max_cnt]['tags'])
            sl = list(t[t['cnt'] == max_cnt]['songs']) 

            tc=Counter([item for sublist in tl for item in sublist]).most_common()
            sc=Counter([item for sublist in sl for item in sublist]).most_common()
            
            #before updt_date인지 체크
            cand_song = list(map(lambda x : x[0], sc))
            song_result += before_updt_date(cand_song,x.updt_date,self.song_meta)

            for i in tc:
                if (i[0] not in x['tags']) & (i[0] not in tag_result):
                    tag_result.append(i[0])
            
            song_result = remove_seen(x.songs, song_result)
            
            if ((len(song_result) >= 100) & (len(tag_result) >= 10)) :
                break

            max_cnt -= 1

        return [tag_result[:10],song_result[:100]]

    def run(self):
        tqdm.pandas()
        
        #song 개수가 3개 미만일 경우만 rec_songs, rec_tags 뽑아내기
        #최종 제출할 때는 val -> test로
        
        val_not = self.test[(self.test.songs.str.len() >= 3)].copy()
        blank = []
        for i in range(len(val_not)):
            blank.append([])
        val_not['rec_songs'] = blank
        val_not['rec_tags'] = blank
        
        val_min = self.test[(self.test.songs.str.len() < 3)].copy()
        val_min['rec'] = val_min.progress_apply(lambda x : self.mk_rec(x), axis = 1)
        val_min['rec_tags'] = val_min.apply(lambda x : x.rec[0] ,axis = 1)
        val_min['rec_songs'] = val_min.apply(lambda x : x.rec[1] ,axis = 1)
        val_min = val_min.drop(columns = 'rec')
        
        self.test = pd.concat([val_not, val_min]).sort_index(ascending=True)
        
        self.res = []
        
        for pid in self.test.index:
            self.res.append({
                    "id": self.test.loc[pid, "id"],
                    "songs": self.test.loc[pid, "rec_songs"],
                    "tags": self.test.loc[pid, "rec_tags"]
            })
        
        write_json(self.res, self.FILE_PATH, "base_results.json")


In [ ]:
FILE_PATH = '/Users/ahrimahn/melon music/kakao arena/'
base = MakeBaselineResults(FILE_PATH)
results = base.run()

/Users/ahrimahn/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
 99%|█████████▊| 2077/2104 [30:54<00:26,  1.01it/s]  

In [14]:
class PlaylistEmbedding:
    def __init__(self, FILE_PATH):
        self.FILE_PATH = FILE_PATH
        self.min_count = 3
        self.size = 100
        self.window = 210 #원래 값 210
        self.sg = 5
        
        self.p2v_model = WordEmbeddingsKeyedVectors(self.size)
        #self.p2v_model = KeyedVectors.load("p2v_model.model")
        #self.w2v_model = Word2Vec.load("word2vec.model")
        
        with open(os.path.join(FILE_PATH, 'train.json'), encoding="utf-8") as f:
            self.train = json.load(f)
        
        with open(os.path.join(FILE_PATH, 'val.json'), encoding="utf-8") as f:
            self.val = json.load(f)
            
        with open(os.path.join(FILE_PATH, 'test.json'), encoding="utf-8") as f:
            self.test = json.load(f)
        
        with open(os.path.join(FILE_PATH, 'song_meta.json'), encoding="utf-8") as f:
            self.song_meta = pd.DataFrame(json.load(f))
        
        with open(os.path.join(FILE_PATH, 'base_results.json'), encoding="utf-8") as f: #baseline results
            self.most_results = json.load(f)
        
    def get_dic(self, train, val, test):
        song_dic = {}
        tag_dic = {}
        
        train = train + val
        data = train + test
        
        for q in tqdm(data):
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']

        
        self.song_dic = song_dic
        self.tag_dic = tag_dic
        total = list(map(lambda x: list(map(str, x['songs'])) + list(x['tags']), data))
        total = [x for x in total if len(x)>1]
        self.total = total
        
        
    def get_w2v(self, total, min_count, size, window, sg):
        w2v_model = Word2Vec(total, min_count = min_count, size = size, window = window, sg = sg, workers = 4)
        self.w2v_model = w2v_model
        w2v_model.save("word2vec.model")
    

    def update_p2v(self, train, val, test, w2v_model):
        ID = []   
        vec = []
        for q in tqdm(train + val + test):
            tmp_vec = 0
            
            #add_tags를 train에는 안넣고 추천 때만 넣는다??
            songs_and_tags = q['songs'] + q['tags']
            
            
            if len(songs_and_tags)>=1:
                for song in songs_and_tags:
                    try:
                        tmp_vec += w2v_model.wv.get_vector(str(song))
                    except KeyError:
                        pass
            if type(tmp_vec)!=int:
                ID.append(str(q['id']))
                vec.append(tmp_vec)
        self.p2v_model.add(ID, vec)
        self.p2v_model.save('p2v_model.model')
    
    def popular_results (self,train) :
        tags_dict = {}
        songs_dict = {}
        
        for i in tqdm(train):
            for j in i['tags']:
                if j in tags_dict:
                    tags_dict[j]+= 1
                else:
                    tags_dict[j] = 1
            for j in i['songs']:
                if j in songs_dict:
                    songs_dict[j] += 1
                else:
                    songs_dict[j] = 1

        self.popular_tags = list(map(lambda x: x[0],sorted(tags_dict.items(), reverse=True,key=lambda item: item[1])[:10]))
        self.popular_songs = list(map(lambda x: x[0],sorted(songs_dict.items(), reverse=True,key=lambda item: item[1])[:100]))
    
    def get_result(self, p2v_model, song_dic, tag_dic, most_results, val):
        answers = []

        for n, q in tqdm(enumerate(val), total = len(val)):
            try:
                cand_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=200)]
                
                #similarity 기준치 이상인 플레이리스트로만!
                
                #similarity 가 1에 가까운 플레이리스트는 모든 노래를 추천??!
                
                get_song = []
                get_tag = []
                cand_song = []
                
                for ID in cand_id:
                    get_song += song_dic[ID]
                    get_tag += tag_dic[ID]
                
                cand_song += list(pd.value_counts(get_song)[:1000].index)    
                
                get_song = before_updt_date(cand_song,q['updt_date'],self.song_meta) #before_updt_date 모듈 추가!
                
                get_tag = list(pd.value_counts(get_tag)[:10].index)
                
                answers.append({
                    "id": q["id"],
                    "songs": remove_seen(q["songs"], get_song)[:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
                
            except: #키에러가 나는 이유는?! 
                answers.append({
                  "id": most_results[n]["id"],
                  "songs": most_results[n]['songs'],
                  "tags": most_results[n]["tags"],
                })

        # check and update answer
        
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results[n]['songs'])[:100-len(q['songs'])]
            
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results[n]['tags'])[:10-len(q['tags'])]
            
        self.answers = answers
    
    def run(self):
        self.get_dic(self.train, self.val, self.test)
        self.get_w2v(self.total, self.min_count, self.size, self.window, self.sg)
        self.update_p2v(self.train, self.val, self.test, self.w2v_model)
        self.get_result(self.p2v_model, self.song_dic, self.tag_dic, self.most_results, self.test)
        
        write_json(self.answers,self.FILE_PATH, 'results.json')
        return (self.answers)


In [15]:
import time
start = time.time()
FILE_PATH = '/Users/ahrimahn/melon music/kakao arena/'
U_space = PlaylistEmbedding(FILE_PATH)
U_space.run()
print("실행시간 :",time.time()-start)

100%|██████████| 10740/10740 [02:34<00:00, 69.38it/s]


실행시간 : 1037.623545885086


In [38]:
results

(['기분전환', '감성', '휴식', '발라드', '잔잔한', '드라이브', '힐링', '사랑', '새벽', '밤'],
 [144663,
  116573,
  357367,
  366786,
  654757,
  133143,
  349492,
  675115,
  463173,
  42155,
  610933,
  396828,
  461341,
  174749,
  520093,
  701557,
  549178,
  485155,
  650494,
  523521,
  13281,
  648628,
  449244,
  680366,
  169984,
  422915,
  11657,
  418935,
  187047,
  547967,
  422077,
  350309,
  627363,
  625875,
  300087,
  132994,
  215411,
  427724,
  442014,
  668128,
  582252,
  663256,
  253755,
  643628,
  448116,
  339802,
  581799,
  348200,
  26083,
  37748,
  341513,
  505036,
  199262,
  407828,
  105140,
  68348,
  140867,
  235773,
  209993,
  209135,
  339124,
  487911,
  493762,
  672550,
  509998,
  531820,
  27469,
  157055,
  519391,
  473514,
  232874,
  75842,
  117595,
  446812,
  295250,
  152422,
  224921,
  678762,
  351342,
  15318,
  377243,
  146989,
  246531,
  205179,
  645489,
  108004,
  464051,
  13198,
  302646,
  152475,
  343974,
  236393,
  95323,
  640657,
  4